## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, Dense, Activation, Dropout

# Cifar 10 資料( 參考 [Day_015_HW](https://github.com/YenLinWu/DL_CVMarathon/blob/master/%E6%AF%8F%E6%97%A5%E4%BD%9C%E6%A5%AD/Day_015_HW.ipynb) )

In [2]:
( x_train, y_train ), ( x_test, y_test ) = cifar10.load_data()

print( f'Shape of Training Data = {x_train.shape}' ) 

# Normalize Data
def normalize( X_train, X_test ):
    mean = np.mean( X_train, axis=(0,1,2,3) )
    std = np.std( X_train, axis=(0,1,2,3) )
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7) 
    return X_train, X_test, mean, std
    
# Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize( x_train, x_test ) 

# OneHot Label
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform( y_train ).toarray()
y_test = one_hot.transform( y_test ).toarray()

Shape of Training Data = (50000, 32, 32, 3)


# Xception 遷移式學習

In [3]:
from tensorflow.keras.applications.xception import Xception

input_tensor = Input( shape=(32,32,3) )

Base_Model = Xception( include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000 )

In [4]:
x = Base_Model.output
x = GlobalAveragePooling2D()(x)
x = Dense( 128, activation='relu', name='Dense' )(x)
x = BatchNormalization( name='BN' )(x)
Output_Layer = Dense( 10, activation='softmax', name='Softmax' )(x)
Transfer_Model = Model( inputs=Base_Model.input, outputs=Output_Layer, name='Transfer_Model' )

print( 'Numbers of Layer of Transfer Model = ', len(Transfer_Model.layers) )
Transfer_Model.summary()

Numbers of Layer of Transfer Model =  136
Model: "Transfer_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
___________________________________________

## 鎖定特定幾層不要更新權重

In [5]:
for layer in Transfer_Model.layers[:100]:
    layer.trainable = False
for layer in Transfer_Model.layers[100:]:
    layer.trainable = True

for i, layer in enumerate( Transfer_Model.layers ):
    print( i, layer.name, layer.trainable )

0 input_2 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d False
13 block2_pool False
14 batch_normalization False
15 add False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_1 False
23 block3_pool False
24 batch_normalization_1 False
25 add_1 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_2 False
33 block4_pool False
34 batch_normalization_2 False
35 add_2 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2 False

# 訓練模型

In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
Transfer_Model.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )
Transfer_Model.fit( x_train, y_train, batch_size=32, epochs=30 )

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 36s 718us/sample - loss: 1.5946 - accuracy: 0.4242
Epoch 2/30
50000/50000 [==============================] - 30s 590us/sample - loss: 1.0840 - accuracy: 0.6280
Epoch 3/30
50000/50000 [==============================] - 30s 591us/sample - loss: 0.9423 - accuracy: 0.6788
Epoch 4/30
50000/50000 [==============================] - 30s 594us/sample - loss: 0.8493 - accuracy: 0.7069
Epoch 5/30
50000/50000 [==============================] - 30s 592us/sample - loss: 0.7655 - accuracy: 0.7375
Epoch 6/30
50000/50000 [==============================] - 30s 592us/sample - loss: 0.6875 - accuracy: 0.7629
Epoch 7/30
50000/50000 [==============================] - 30s 603us/sample - loss: 0.6114 - accuracy: 0.7888
Epoch 8/30
50000/50000 [==============================] - 30s 594us/sample - loss: 0.5421 - accuracy: 0.8122
Epoch 9/30
50000/50000 [==============================] - 30s 595us/sample - loss: 0.4821 - accuracy: 0.8